In [1]:
# to use my GPU
# !pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118


from datasets import concatenate_datasets, load_dataset, Audio, DatasetDict
from transformers import AutoFeatureExtractor, Wav2Vec2FeatureExtractor, AutoModelForAudioClassification, WhisperForAudioClassification, WhisperProcessor,  TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
from random import randint

import torch
import evaluate
from torch.cuda import device_count
import os
import librosa
import pandas as pd
from pathlib import Path


In [2]:

model_name = "distil-whisper/distil-large-v3" # "whisper-large-v3_ADReSSO"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(
    model_name, num_labels=2, ignore_mismatched_sizes=True
)
del model.config.__dict__["max_length"]
del model.config.__dict__["suppress_tokens"]
del model.config.__dict__["begin_suppress_tokens"]
model.to(device)
model.save_pretrained("model")

# model1 = WhisperForAudioClassification.from_pretrained(model_name, num_labels=2)
# processor1 = WhisperProcessor.from_pretrained(model_name)

preprocess = lambda examples: feature_extractor(
    [i["array"][(n := randint(0, len(i["array"]) - (m := min(len(i["array"]), feature_extractor.sampling_rate*30)))) : n + m] for i in examples["audio"]],
    sampling_rate=feature_extractor.sampling_rate,
    do_normalize=True,
    # max_length=16_000*args.sample_duration,
    # truncation=True,
)

#### LOAD DATASET HERE ############
AD_PATH = 'ADReSSo21/diagnosis/train/audio/ad'
CN_PATH = 'ADReSSo21/diagnosis/train/audio/cn'

ad_dataset = (
    load_dataset(AD_PATH)
    .cast_column("audio", Audio(sampling_rate=feature_extractor.sampling_rate))
)
ad_dataset = ad_dataset.map(lambda example: {"label": 0})

cn_dataset = (
    load_dataset(CN_PATH)
    .cast_column("audio", Audio(sampling_rate=feature_extractor.sampling_rate))
)
cn_dataset = cn_dataset.map(lambda example: {"label": 1})

dataset = concatenate_datasets([ad_dataset["train"], cn_dataset["train"]])
dataset = DatasetDict({
    "train": dataset
})
dataset["train"], dataset["valid"] = dataset["train"].train_test_split(0.25).values()
# dataset["valid"], dataset["test"] = dataset["valid"].train_test_split(0.2).values()
dataset = dataset.map(preprocess, remove_columns="audio", batched=True)


Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at distil-whisper/distil-large-v3 and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Resolving data files:   0%|          | 0/87 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/79 [00:00<?, ?it/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

Exception ignored from cffi callback <function SoundFile._init_virtual_io.<locals>.vio_read at 0x000001CE950EADE0>:
Traceback (most recent call last):
  File "c:\Users\qscre\miniconda3\envs\aml\Lib\site-packages\soundfile.py", line 1246, in vio_read
    data_read = file.readinto(buf)
                ^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


Map:   0%|          | 0/42 [00:00<?, ? examples/s]

In [3]:
# Convert dataset to Tensor (Data must be in PyTorch format, eg: {"input_values": audio_tensor, "label": label_tensor})
# class AudioDataset(Dataset):
#     def __init__(self, audio_data, labels):
#         self.audio_data = audio_data
#         self.labels = labels

#     def __len__(self):
#         return len(self.audio_data)

#     def __getitem__(self, idx):
#         # Convert to tensor and return
#         audio_tensor = torch.tensor(self.audio_data[idx]).float()  # Make sure it's float
#         label_tensor = torch.tensor(self.labels[idx]).long()  # Make sure it's long for classification
#         return {"input_values": audio_tensor, "label": label_tensor}
    
# train_dataset = {"input_features": [torch.tensor(feature, dtype=torch.float) for feature in dataset["train"]["input_features"]], 
#                  "label": [torch.tensor(feature, dtype=torch.float) for feature in dataset["train"]["label"]]}
# valid_dataset = {"input_features": [torch.tensor(feature, dtype=torch.float) for feature in dataset["valid"]["input_features"]], 
#                  "label": [torch.tensor(feature, dtype=torch.float) for feature in dataset["valid"]["label"]]}

train_dataset = dataset["train"].with_format("torch")
val_dataset = dataset["valid"].with_format("torch")
# test_dataset = dataset["test"].with_format("torch")


In [4]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
specificity = evaluate.load("nevikw39/specificity")

## training part
training_args = TrainingArguments(
    output_dir="models/whisper-large-v3_ADReSSo" + ("_fp16" if False else ""),
    fp16=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_train_batch_size=2, # 8
    per_device_eval_batch_size=2,# 8
    gradient_accumulation_steps=4,
    # gradient_checkpointing=True,
    num_train_epochs=10, # 100
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,

)

trainer = Trainer (
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, # dataset["valid"],
    # eval_dataset=encoded_dataset["test"].select(np.random.choice(71, 42)),
    tokenizer=feature_extractor,
    compute_metrics=lambda eval_pred: (
        accuracy.compute(
            predictions=(pred := np.argmax(eval_pred.predictions, axis=1)),
            references=eval_pred.label_ids,
        ) | f1.compute(
            predictions=pred,
            references=eval_pred.label_ids,
        ) | specificity.compute(
            predictions=pred,
            references=eval_pred.label_ids,
        )
    ),
    callbacks=[EarlyStoppingCallback(10)],
)

#trainer.train()



c:\Users\qscre\miniconda3\envs\aml\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/150 [00:00<?, ?it/s]

c:\Users\qscre\miniconda3\envs\aml\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:599: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.7083, 'grad_norm': 3.1401898860931396, 'learning_rate': 1.9999999999999998e-05, 'epoch': 0.65}


  0%|          | 0/21 [00:00<?, ?it/s]

...

  0%|          | 0/21 [00:00<?, ?it/s]

{'eval_loss': 1.361391544342041, 'eval_accuracy': 0.7142857142857143, 'eval_f1': 0.6666666666666666, 'eval_specificity': 0.8181818181818182, 'eval_runtime': 196.6871, 'eval_samples_per_second': 0.214, 'eval_steps_per_second': 0.107, 'epoch': 8.97}
{'loss': 0.0012, 'grad_norm': 0.03122571110725403, 'learning_rate': 2.222222222222222e-06, 'epoch': 9.03}
{'loss': 0.0012, 'grad_norm': 0.02977282926440239, 'learning_rate': 0.0, 'epoch': 9.68}


  0%|          | 0/21 [00:00<?, ?it/s]

{'eval_loss': 1.3726603984832764, 'eval_accuracy': 0.7142857142857143, 'eval_f1': 0.6666666666666666, 'eval_specificity': 0.8181818181818182, 'eval_runtime': 198.9486, 'eval_samples_per_second': 0.211, 'eval_steps_per_second': 0.106, 'epoch': 9.68}
{'train_runtime': 11724.7202, 'train_samples_per_second': 0.106, 'train_steps_per_second': 0.013, 'train_loss': 0.21776453108216326, 'epoch': 9.68}


TrainOutput(global_step=150, training_loss=0.21776453108216326, metrics={'train_runtime': 11724.7202, 'train_samples_per_second': 0.106, 'train_steps_per_second': 0.013, 'total_flos': 1.756691463168e+18, 'train_loss': 0.21776453108216326, 'epoch': 9.67741935483871})

In [5]:
# print("test_dataset", test_dataset)

# eval_results = trainer.evaluate(test_dataset)

# eval_results

In [8]:

# Evaluate the model on the evaluation dataset and capture the evaluation results
eval_result = trainer.evaluate()

# Print the evaluation metrics
print("Evaluation results:", eval_result)

  0%|          | 0/21 [00:00<?, ?it/s]

KeyboardInterrupt: 